In [1]:
import pandas as pd, numpy as np
import vivarium_output_loader as vol
import lsff_output_processing as lop
import lsff_summarizer
from lsff_summarizer import LSFFOutputSummarizer

!whoami
!date

ndbs
Thu Apr 23 14:55:52 PDT 2020


In [2]:
%load_ext autoreload
%autoreload 2

## Define directories

In [22]:
base_directory = '/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/'

vad_path = base_directory + 'vad_prevalence_in_outputs/india_4_22'
ntd_path = base_directory + 'ntd_state_persontime_in_results/india_4_23'

# models_modelruns = {
#     'vad_prevalence_in_outputs': 'india_4_22',
#     'ntd_state_persontime_in_results': 'india_4_23'
# }

In [17]:
# vad_directory = ('/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/'
#                 + 'vad_prevalence_in_outputs/india_4_22'
#                  )
# ntd_directory = ('/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/'
#                  'ntd_state_persontime_in_results/india_4_23'
#                 )
#     

# Who knew? You can concatenate strings without using +
x = 'a' 'b'
x

'ab'

In [18]:
# But this doesn't work, i.e. you can only use
# string literals, not variables:
# y = x 'c'

## Load `output.hdf` files

In [23]:
vad_raw_output = pd.read_hdf(f'{vad_path}/output.hdf')
vad_raw_output.head()
# raw_output = vol.load_by_location_and_rundate(base_directory, models_modelruns)
# raw_output.head()

total_population_untracked  \
input_draw_number random_seed                               
946.0             35.0                            10510.0   
357.0             35.0                            10446.0   
946.0             21.0                            10478.0   
357.0             45.0                            10526.0   
                  69.0                            10492.0   

                               total_population_tracked  total_population  \
input_draw_number random_seed                                               
946.0             35.0                          10048.0           20558.0   
357.0             35.0                           9847.0           20293.0   
946.0             21.0                           9771.0           20249.0   
357.0             45.0                           9882.0           20408.0   
                  69.0                           9907.0           20399.0   

                               diarrheal_diseases_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                                  
946.0             35.0                                                2330.0   
357.0             35.0                                                2192.0   
946.0             21.0                                                2190.0   
357.0             45.0                                                2209.0   
                  69.0                                                2078.0   

                               susceptible_to_diarrheal_diseases_event_count  \
input_draw_number random_seed                                                  
946.0             35.0                                                9990.0   
357.0             35.0                                                9713.0   
946.0             21.0                                                9727.0   
357.0             45.0                                                9830.0   
                  69.0                                                9248.0   

                               diarrheal_diseases_event_count  \
input_draw_number random_seed                                   
946.0             35.0                                12967.0   
357.0             35.0                                12624.0   
946.0             21.0                                12555.0   
357.0             45.0                                12723.0   
                  69.0                                11947.0   

                               measles_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                       
946.0             35.0                                      786.0   
357.0             35.0                                      345.0   
946.0             21.0                                      733.0   
357.0             45.0                                      312.0   
                  69.0                                      272.0   

                               susceptible_to_measles_event_count  \
input_draw_number random_seed                                       
946.0             35.0                                        0.0   
357.0             35.0                                        0.0   
946.0             21.0                                        0.0   
357.0             45.0                                        0.0   
                  69.0                                        0.0   

                               measles_event_count  \
input_draw_number random_seed                        
946.0             35.0                       882.0   
357.0             35.0                       396.0   
946.0             21.0                       849.0   
357.0             45.0                       357.0   
                  69.0                       312.0   

                               recovered_from_measles_event_count  ...  \
input_draw_number random_seed                                   

In [24]:
ntd_raw_output = pd.read_hdf(f'{ntd_path}/output.hdf')
ntd_raw_output.head()

total_population_untracked  \
input_draw_number random_seed                               
357.0             44.0                            10430.0   
946.0             35.0                            10510.0   
357.0             69.0                            10471.0   
602.0             21.0                            10440.0   
                  69.0                            10511.0   

                               total_population_tracked  total_population  \
input_draw_number random_seed                                               
357.0             44.0                           9923.0           20353.0   
946.0             35.0                          10048.0           20558.0   
357.0             69.0                           9928.0           20399.0   
602.0             21.0                           9901.0           20341.0   
                  69.0                          10000.0           20511.0   

                               diarrheal_diseases_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                                  
357.0             44.0                                                2214.0   
946.0             35.0                                                2330.0   
357.0             69.0                                                2260.0   
602.0             21.0                                                2094.0   
                  69.0                                                2239.0   

                               susceptible_to_diarrheal_diseases_event_count  \
input_draw_number random_seed                                                  
357.0             44.0                                                9617.0   
946.0             35.0                                                9990.0   
357.0             69.0                                                9872.0   
602.0             21.0                                                9463.0   
                  69.0                                                9642.0   

                               diarrheal_diseases_event_count  \
input_draw_number random_seed                                   
357.0             44.0                                12481.0   
946.0             35.0                                12967.0   
357.0             69.0                                12780.0   
602.0             21.0                                12305.0   
                  69.0                                12558.0   

                               measles_prevalent_cases_at_sim_end  \
input_draw_number random_seed                                       
357.0             44.0                                      332.0   
946.0             35.0                                      786.0   
357.0             69.0                                      329.0   
602.0             21.0                                     1520.0   
                  69.0                                     1567.0   

                               susceptible_to_measles_event_count  \
input_draw_number random_seed                                       
357.0             44.0                                        0.0   
946.0             35.0                                        0.0   
357.0             69.0                                        0.0   
602.0             21.0                                        0.0   
                  69.0                                        0.0   

                               measles_event_count  \
input_draw_number random_seed                        
357.0             44.0                       378.0   
946.0             35.0                       882.0   
357.0             69.0                       375.0   
602.0             21.0                      1713.0   
                  69.0                      1755.0   

                               recovered_from_measles_event_count  ...  \
input_draw_number random_seed                                   

## Load count space data

In [26]:
vad_data = vol.load_transformed_count_data(f'{vad_path}/count_data')
vad_data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

In [27]:
ntd_data = vol.load_transformed_count_data(f'{ntd_path}/count_data')
ntd_data.keys()

dict_keys(['gestational_age', 'transition_count', 'deaths', 'state_person_time', 'births_with_ntd', 'population', 'person_time', 'ylls', 'ylds', 'births', 'birth_weight'])

## Print column reports for raw output

In [29]:
vad_output = LSFFOutputSummarizer(vad_raw_output)
vad_output.print_column_report()

Number of data columns in output: 14501
Total number of columns captured in categories: 14501

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 0, 'intervention': 1, 'run_time': 0, 'diseases_at_end': 4, 'transition_count': 3465, 'population': 5, 'person_time': 432, 'state_person_time': 3456, 'treated_days': 0, 'mortality': 2160, 'total_daly': 2, 'yld': 2592, 'yll': 2160, 'categorical_risk': 0, 'birth_prevalence': 108, 'live_births': 108, 'proportion': 2, 'distribution_mean': 2, 'distribution_std_dev': 2, 'prevalence': 0} 

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (5 categories with no matching data columns):
 ['location', 'run_time', 'treated_days', 'categorical_risk', 'prevalence']


In [30]:
ntd_output = LSFFOutputSummarizer(ntd_raw_output)
ntd_output.print_column_report()

Number of data columns in output: 15365
Total number of columns captured in categories: 15365

Number of columns in each category:
 {'input_draw': 1, 'random_seed': 1, 'location': 0, 'intervention': 1, 'run_time': 0, 'diseases_at_end': 4, 'transition_count': 3465, 'population': 5, 'person_time': 432, 'state_person_time': 4320, 'treated_days': 0, 'mortality': 2160, 'total_daly': 2, 'yld': 2592, 'yll': 2160, 'categorical_risk': 0, 'birth_prevalence': 108, 'live_births': 108, 'proportion': 2, 'distribution_mean': 2, 'distribution_std_dev': 2, 'prevalence': 0} 

Missing (0 data column(s) not captured in a category):
 set()

Repeated (0 data column(s) appearing in more than one category):
 {}

Empty categories (5 categories with no matching data columns):
 ['location', 'run_time', 'treated_days', 'categorical_risk', 'prevalence']


## Check state person time columns for both runs

It looks like this is the only category where there's a difference

In [31]:
# Total number of non-state-person-time columns for the two models is equal
(15365 - 4320) == (14501 - 3456)

True

In [45]:
# There are 864 more columns in the NTD output (all in state person time)
4320 - 3456

864

In [63]:
# 864 = 2^5 * 3^3
864/4/4/2/3/3/3

1.0

In [67]:
2**5 * 3**3

864

In [80]:
# 2 NTD states x 5 years x 2 sexes x 4 age groups x 3 FA coverage groups x 3 VA coverage groups
2*5*2*4*3*3

720

So there are 144 extra columns...

In [76]:
864-720

144

### Try to find the 144 columns unaccounted for in `ntd_output`

Ahh, there are 6 years, not 5

In [78]:
ntd_output.subdata('state_person_time').filter(regex='neural_tube_defects').head()

susceptible_to_neural_tube_defects_person_time_in_2020_among_male_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                         
357.0             44.0                                                  4.982888                                                                                      
946.0             35.0                                                  2.989733                                                                                      
357.0             69.0                                                  4.982888                                                                                      
602.0             21.0                                                  5.481177                                                                                      
                  69.0                                                  6.976044                                                                                      

                               susceptible_to_neural_tube_defects_person_time_in_2020_among_female_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                           
357.0             44.0                                                  5.481177                                                                                        
946.0             35.0                                                  3.986311                                                                                        
357.0             69.0                                                  4.484600                                                                                        
602.0             21.0                                                  4.484600                                                                                        
                  69.0                                                  3.986311                                                                                        

                               susceptible_to_neural_tube_defects_person_time_in_2020_among_male_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                        
357.0             44.0                                                 14.450376                                                                                     
946.0             35.0                                                 24.416153                                                                                     
357.0             69.0                                                 23.419576                                                                                     
602.0             21.0                                                 19.433265                                                                                     
                  69.0                                                 24.416153                                                                                     

                               susceptible_to_neural_tube_defects_person_time_in_2020_among_female_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                          
357.0             44.0                                                 17.938398                                                                                       
946.0     

Aha! The raw output includes 2025, so there are 6 years, not 5

In [81]:
# 2 NTD states x 6 years x 2 sexes x 4 age groups x 3 FA coverage groups x 3 VA coverage groups
2*6*2*4*3*3

864

In [33]:
vad_output.columns('state_person_time')[-10:]

['susceptible_to_vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'susceptible_to_vitamin_a_deficiency_person_time_in_2025_among_female_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_female_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_female_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_male_in_age_group_post_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'vitamin_a_deficiency_person_time_in_2025_among_fem

In [34]:
ntd_output.columns('state_person_time')[-10:]

['susceptible_to_neural_tube_defects_person_time_in_2025_among_male_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'susceptible_to_neural_tube_defects_person_time_in_2025_among_female_in_age_group_1_to_4_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_male_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_female_in_age_group_early_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_male_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_female_in_age_group_late_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_male_in_age_group_post_neonatal_folic_acid_covered_vitamin_a_effectively_covered',
 'neural_tube_defects_person_time_in_2025_among_female_in_a

In [42]:
vad_output.subdata('state_person_time').head()

susceptible_to_diarrheal_diseases_person_time_in_2020_among_male_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                        
946.0             35.0                                                  2.989733                                                                                     
357.0             35.0                                                  7.474333                                                                                     
946.0             21.0                                                  9.467488                                                                                     
357.0             45.0                                                  7.474333                                                                                     
                  69.0                                                  0.996578                                                                                     

                               susceptible_to_diarrheal_diseases_person_time_in_2020_among_female_in_age_group_early_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                          
946.0             35.0                                                  3.488022                                                                                       
357.0             35.0                                                 11.958932                                                                                       
946.0             21.0                                                  5.481177                                                                                       
357.0             45.0                                                  9.965777                                                                                       
                  69.0                                                  0.498289                                                                                       

                               susceptible_to_diarrheal_diseases_person_time_in_2020_among_male_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                       
946.0             35.0                                                 24.416153                                                                                    
357.0             35.0                                                 20.429843                                                                                    
946.0             21.0                                                 23.917864                                                                                    
357.0             45.0                                                 22.422998                                                                                    
                  69.0                                                  3.986311                                                                                    

                               susceptible_to_diarrheal_diseases_person_time_in_2020_among_female_in_age_group_late_neonatal_folic_acid_unknown_vitamin_a_uncovered  \
input_draw_number random_seed                                                                                                                                         
946.0             35.0                                                 19.931554                                                                                      
357.0             35.0            

## See what the scenario columns look like

In [36]:
vad_output.subdata('intervention').head(10)

fortification_intervention.scenario
input_draw_number random_seed                                    
946.0             35.0                                   baseline
357.0             35.0                                   baseline
946.0             21.0                                   baseline
357.0             45.0          folic_acid_fortification_scale_up
                  69.0           vitamin_a_fortification_scale_up
946.0             69.0           vitamin_a_fortification_scale_up
357.0             12.0                                   baseline
602.0             35.0          folic_acid_fortification_scale_up
946.0             12.0           vitamin_a_fortification_scale_up
                  22.0          folic_acid_fortification_scale_up

In [37]:
ntd_output.subdata('intervention').head(10)

fortification_intervention.scenario
input_draw_number random_seed                                    
357.0             44.0                                   baseline
946.0             35.0          folic_acid_fortification_scale_up
357.0             69.0          folic_acid_fortification_scale_up
602.0             21.0                                   baseline
                  69.0                                   baseline
946.0             22.0          folic_acid_fortification_scale_up
                  22.0           vitamin_a_fortification_scale_up
602.0             22.0          folic_acid_fortification_scale_up
357.0             45.0                                   baseline
946.0             45.0                                   baseline

## Load state person time from count data

Ok, so it looks like the difference is that state person time for neural tube defects has been added in the second model, which is what we expect.

In [40]:
vad_spt = vad_data['state_person_time']
vad_spt.head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


In [64]:
vad_spt.shape

(25920, 10)

In [83]:
# Note there are only 5 years in count space data, but 6 in the raw output
print([vad_spt[col].nunique() for col in vad_spt.columns])

[5, 4, 2, 8, 3, 3, 1, 3, 3, 4025]


In [86]:
# Check that product of identification columns equals total length of dataframe
prod = 1
for col in vad_spt.columns[:-1]:
    prod *= vad_spt[col].nunique()
prod

25920

In [73]:
# 8 cause states, 3 input draws
5*4*2*8*3*3*1*3*3

25920

In [41]:
ntd_spt = ntd_data['state_person_time']
ntd_spt.head()

,year,age_group,sex,cause,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,diarrheal_diseases,covered,covered,person_time,602,folic_acid_fortification_scale_up,0.0


In [65]:
ntd_spt.shape

(32400, 10)

In [72]:
# 10 cause states, 3 input draws
5*4*2*10*3*3*1*3*3

32400

In [75]:
5*4*2*10*3*3 - 5*4*2*8*3*3

720

In [43]:
vad_spt.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_vitamin_a_deficiency',
       'vitamin_a_deficiency'], dtype=object)

In [44]:
ntd_spt.cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_neural_tube_defects',
       'susceptible_to_vitamin_a_deficiency', 'vitamin_a_deficiency'],
      dtype=object)

In [50]:
ntd_spt.vitamin_a_fortification_group.unique()

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

In [51]:
ntd_spt.folic_acid_fortification_group.unique()

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [52]:
ntd_spt.age_group.unique()

array(['1_to_4', 'early_neonatal', 'late_neonatal', 'post_neonatal'],
      dtype=object)

In [54]:
ntd_spt.year.unique()

array(['2020', '2021', '2022', '2023', '2024'], dtype=object)

In [57]:
ntd_spt.query('folic_acid_fortification_group == "unknown"').cause.unique()

array(['diarrheal_diseases', 'lower_respiratory_infections', 'measles',
       'neural_tube_defects', 'susceptible_to_diarrheal_diseases',
       'susceptible_to_lower_respiratory_infections',
       'susceptible_to_measles', 'susceptible_to_neural_tube_defects',
       'susceptible_to_vitamin_a_deficiency', 'vitamin_a_deficiency'],
      dtype=object)

In [87]:
365*6

2190